In [196]:
%pip install -q -U urlextract

In [197]:
import urllib.request
import tarfile
from pathlib import Path

def fetch_spam_data(easy=True):
    ham = "easy_ham" if easy else "hard_ham"

    spam_root = "http://spamassassin.apache.org/old/publiccorpus/"
    ham_url = spam_root + f"20030228_{ham}.tar.bz2"
    spam_url = spam_root + "20030228_spam.tar.bz2"

    spam_path = Path() / "datasets" / "spam"
    spam_path.mkdir(parents=True, exist_ok=True)
    for dir_name, tar_name, url in ((ham, "ham", ham_url),
                                    ("spam", "spam", spam_url)):
        if not (spam_path / dir_name).is_dir():
            path = (spam_path / tar_name).with_suffix(".tar.bz2")
            print("Downloading", path)
            urllib.request.urlretrieve(url, path)
            tar_bz2_file = tarfile.open(path)
            tar_bz2_file.extractall(path=spam_path)
            tar_bz2_file.close()
    return [spam_path / dir_name for dir_name in (ham, "spam")]

ham_dir, spam_dir = fetch_spam_data()
ham_filenames = [f for f in sorted(ham_dir.iterdir()) if len(f.name) > 20]
spam_filenames = [f for f in sorted(spam_dir.iterdir()) if len(f.name) > 20]

# Easy ham

In [198]:
import email
import email.policy

def load_email(filepath):
    with open(filepath, "rb") as f:
        return email.parser.BytesParser(policy=email.policy.default).parse(f)

ham_emails = [load_email(filepath) for filepath in ham_filenames]
spam_emails = [load_email(filepath) for filepath in spam_filenames]

In [199]:
print(ham_emails[1].get_content().strip())

Martin A posted:
Tassos Papadopoulos, the Greek sculptor behind the plan, judged that the
 limestone of Mount Kerdylio, 70 miles east of Salonika and not far from the
 Mount Athos monastic community, was ideal for the patriotic sculpture. 
 
 As well as Alexander's granite features, 240 ft high and 170 ft wide, a
 museum, a restored amphitheatre and car park for admiring crowds are
planned
---------------------
So is this mountain limestone or granite?
If it's limestone, it'll weather pretty fast.

------------------------ Yahoo! Groups Sponsor ---------------------~-->
4 DVDs Free +s&p Join Now
http://us.click.yahoo.com/pt6YBB/NXiEAA/mG3HAA/7gSolB/TM
---------------------------------------------------------------------~->

To unsubscribe from this group, send an email to:
forteana-unsubscribe@egroups.com

 

Your use of Yahoo! Groups is subject to http://docs.yahoo.com/info/terms/


In [200]:
print(spam_emails[6].get_content().strip())

Help wanted.  We are a 14 year old fortune 500 company, that is
growing at a tremendous rate.  We are looking for individuals who
want to work from home.

This is an opportunity to make an excellent income.  No experience
is required.  We will train you.

So if you are looking to be employed from home with a career that has
vast opportunities, then go:

http://www.basetel.com/wealthnow

We are looking for energetic and self motivated people.  If that is you
than click on the link and fill out the form, and one of our
employement specialist will contact you.

To be removed from our link simple go to:

http://www.basetel.com/remove.html


4139vOLW7-758DoDY1425FRhM1-764SMFc8513fCsLl40


In [201]:
def get_email_structure(email):
    if isinstance(email, str):
        return email
    payload = email.get_payload()
    if isinstance(payload, list):
        multipart = ", ".join([get_email_structure(sub_email)
                               for sub_email in payload])
        return f"multipart({multipart})"
    else:
        return email.get_content_type()

In [202]:
from collections import Counter

def structures_counter(emails):
    structures = Counter()
    for email in emails:
        structure = get_email_structure(email)
        structures[structure] += 1
    return structures

In [203]:
structures_counter(ham_emails).most_common()

[('text/plain', 2408),
 ('multipart(text/plain, application/pgp-signature)', 66),
 ('multipart(text/plain, text/html)', 8),
 ('multipart(text/plain, text/plain)', 4),
 ('multipart(text/plain)', 3),
 ('multipart(text/plain, application/octet-stream)', 2),
 ('multipart(text/plain, text/enriched)', 1),
 ('multipart(text/plain, application/ms-tnef, text/plain)', 1),
 ('multipart(multipart(text/plain, text/plain, text/plain), application/pgp-signature)',
  1),
 ('multipart(text/plain, video/mng)', 1),
 ('multipart(text/plain, multipart(text/plain))', 1),
 ('multipart(text/plain, application/x-pkcs7-signature)', 1),
 ('multipart(text/plain, multipart(text/plain, text/plain), text/rfc822-headers)',
  1),
 ('multipart(text/plain, multipart(text/plain, text/plain), multipart(multipart(text/plain, application/x-pkcs7-signature)))',
  1),
 ('multipart(text/plain, application/x-java-applet)', 1)]

In [204]:
structures_counter(spam_emails).most_common()

[('text/plain', 218),
 ('text/html', 183),
 ('multipart(text/plain, text/html)', 45),
 ('multipart(text/html)', 20),
 ('multipart(text/plain)', 19),
 ('multipart(multipart(text/html))', 5),
 ('multipart(text/plain, image/jpeg)', 3),
 ('multipart(text/html, application/octet-stream)', 2),
 ('multipart(text/plain, application/octet-stream)', 1),
 ('multipart(text/html, text/plain)', 1),
 ('multipart(multipart(text/html), application/octet-stream, image/jpeg)', 1),
 ('multipart(multipart(text/plain, text/html), image/gif)', 1),
 ('multipart/alternative', 1)]

In [205]:
for header, value in spam_emails[0].items():
    print(header, ":", value)

Return-Path : <12a1mailbot1@web.de>
Delivered-To : zzzz@localhost.spamassassin.taint.org
Received : from localhost (localhost [127.0.0.1])	by phobos.labs.spamassassin.taint.org (Postfix) with ESMTP id 136B943C32	for <zzzz@localhost>; Thu, 22 Aug 2002 08:17:21 -0400 (EDT)
Received : from mail.webnote.net [193.120.211.219]	by localhost with POP3 (fetchmail-5.9.0)	for zzzz@localhost (single-drop); Thu, 22 Aug 2002 13:17:21 +0100 (IST)
Received : from dd_it7 ([210.97.77.167])	by webnote.net (8.9.3/8.9.3) with ESMTP id NAA04623	for <zzzz@spamassassin.taint.org>; Thu, 22 Aug 2002 13:09:41 +0100
From : 12a1mailbot1@web.de
Received : from r-smtp.korea.com - 203.122.2.197 by dd_it7  with Microsoft SMTPSVC(5.5.1775.675.6);	 Sat, 24 Aug 2002 09:42:10 +0900
To : dcek1a1@netsgo.com
Subject : Life Insurance - Why Pay More?
Date : Wed, 21 Aug 2002 20:31:57 -1600
MIME-Version : 1.0
Message-ID : <0103c1042001882DD_IT7@dd_it7>
Content-Type : text/html; charset="iso-8859-1"
Content-Transfer-Encoding : qu

In [206]:
spam_emails[0]["Subject"]

'Life Insurance - Why Pay More?'

In [207]:
import numpy as np
from sklearn.model_selection import train_test_split

X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails)) # zeros for ham and ones for spam

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [208]:
import re
from html import unescape

def html_to_plain_text(html):
    text = re.sub('<head.*?>.*?</head>', '', html, flags=re.M | re.S | re.I)
    text = re.sub('<a\s.*?>', ' HYPERLINK ', text, flags=re.M | re.S | re.I)
    text = re.sub('<.*?>', '', text, flags=re.M | re.S)
    text = re.sub(r'(\s*\n)+', '\n', text, flags=re.M | re.S)
    return unescape(text)

In [209]:
html_spam_emails = [email for email in X_train[y_train==1]
                    if get_email_structure(email) == "text/html"]
sample_html_spam = html_spam_emails[7]
print(sample_html_spam.get_content().strip()[:1000], "...")

<HTML><HEAD><TITLE></TITLE><META http-equiv="Content-Type" content="text/html; charset=windows-1252"><STYLE>A:link {TEX-DECORATION: none}A:active {TEXT-DECORATION: none}A:visited {TEXT-DECORATION: none}A:hover {COLOR: #0033ff; TEXT-DECORATION: underline}</STYLE><META content="MSHTML 6.00.2713.1100" name="GENERATOR"></HEAD>
<BODY text="#000000" vLink="#0033ff" link="#0033ff" bgColor="#CCCC99"><TABLE borderColor="#660000" cellSpacing="0" cellPadding="0" border="0" width="100%"><TR><TD bgColor="#CCCC99" valign="top" colspan="2" height="27">
<font size="6" face="Arial, Helvetica, sans-serif" color="#660000">
<b>OTC</b></font></TD></TR><TR><TD height="2" bgcolor="#6a694f">
<font size="5" face="Times New Roman, Times, serif" color="#FFFFFF">
<b>&nbsp;Newsletter</b></font></TD><TD height="2" bgcolor="#6a694f"><div align="right"><font color="#FFFFFF">
<b>Discover Tomorrow's Winners&nbsp;</b></font></div></TD></TR><TR><TD height="25" colspan="2" bgcolor="#CCCC99"><table width="100%" border="0" 

In [210]:
print(html_to_plain_text(sample_html_spam.get_content())[:1000], "...")


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Watch for analyst "Strong Buy Recommendations" and several advisory newsletters picking CBYI.  CBYI has filed to be traded on the OTCBB, share prices historically INCREASE when companies get listed on this larger trading exchange. CBYI is trading around 25 cents and should skyrocket to $2.66 - $3.25 a share in the near future.
Put CBYI on your watch list, acquire a position TODAY.
REASONS TO INVEST IN CBYI
A profitable company and is on track to beat ALL earnings estimates!
One of the FASTEST growing distributors in environmental & safety equipment instruments.
Excellent management team, several EXCLUSIVE contracts.  IMPRESSIVE client list including the U.S. Air Force, Anheuser-Busch, Chevron Refining and Mitsubishi Heavy Industries, GE-Energy & Environmental Research.
RAPIDLY GROWING INDUSTRY
Industry revenues exceed $900 million, estimates indicate that there could be as much as $25 billi

In [211]:
def email_to_text(email):
    html = None
    for part in email.walk():
        ctype = part.get_content_type()
        if not ctype in ("text/plain", "text/html"):
            continue
        try:
            content = part.get_content()
        except: # in case of encoding issues
            content = str(part.get_payload())
        if ctype == "text/plain":
            return content
        else:
            html = content
    if html:
        return html_to_plain_text(html)

In [212]:
print(email_to_text(sample_html_spam)[:100], "...")


OTC
 Newsletter
Discover Tomorrow's Winners 
For Immediate Release
Cal-Bay (Stock Symbol: CBYI)
Wat ...


In [213]:
import nltk

stemmer = nltk.PorterStemmer()
for word in ("Computations", "Computation", "Computing", "Computed", "Compute",
             "Compulsive"):
    print(word, "=>", stemmer.stem(word))

Computations => comput
Computation => comput
Computing => comput
Computed => comput
Compute => comput
Compulsive => compuls


In [214]:
import urlextract # may require an Internet connection to download root domain
                  # names

url_extractor = urlextract.URLExtract()
some_text = "Will it detect github.com and https://youtu.be/7Pq-S557XQU?t=3m32s"
print(url_extractor.find_urls(some_text))

['github.com', 'https://youtu.be/7Pq-S557XQU?t=3m32s']


In [215]:
from sklearn.base import BaseEstimator, TransformerMixin

class EmailToWordCounterTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, strip_headers=True, lower_case=True, remove_punctuation=True, replace_urls=True, replace_numbers=True, stemming=True):
        self.strip_headers = strip_headers
        self.lower_case = lower_case
        self.remove_punctuation = remove_punctuation
        self.replace_urls = replace_urls
        self.replace_numbers = replace_numbers
        self.stemming = stemming

  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    X_transformed = []
    for email in X:
      text = email_to_text(email) or ""

      if self.lower_case:
        text.lower()

      if self.replace_urls and url_extractor is not None:
        urls = list(set(url_extractor.find_urls(text)))

        for url in urls:
          text = text.replace(url, " URL ")

      if self.replace_numbers:
        text = re.sub(r'\d+(?:\.\d*)?(?:[eE][+-]?\d+)?', 'NUMBER', text)

      if self.remove_punctuation:
        text = re.sub(r'\W+', ' ', text, flags=re.M)

      word_counts = Counter(text.split())

      if self.stemming and stemmer is not None:
        stemmed_word_counts = Counter()
        for word, count in word_counts.items():
          stemmed_word = stemmer.stem(word)
          stemmed_word_counts[stemmed_word] += count

        word_counts = stemmed_word_counts
      X_transformed.append(word_counts)
    return np.array(X_transformed)

In [216]:
X_few = X_train[:3]
X_few_wordcounts = EmailToWordCounterTransformer().fit_transform(X_few)
X_few_wordcounts

array([Counter({'chuck': 1, 'murcko': 1, 'wrote': 1, 'stuff': 1, 'yawn': 1, 'r': 1}),
       Counter({'the': 11, 'of': 9, 'and': 8, 'all': 3, 'christian': 3, 'to': 3, 'by': 3, 'jefferson': 2, 'i': 2, 'have': 2, 'superstit': 2, 'one': 2, 'on': 2, 'been': 2, 'ha': 2, 'half': 2, 'rogueri': 2, 'teach': 2, 'jesu': 2, 'some': 1, 'interest': 1, 'quot': 1, 'url': 1, 'thoma': 1, 'examin': 1, 'known': 1, 'word': 1, 'do': 1, 'not': 1, 'find': 1, 'in': 1, 'our': 1, 'particular': 1, 'redeem': 1, 'featur': 1, 'they': 1, 'are': 1, 'alik': 1, 'found': 1, 'fabl': 1, 'mytholog': 1, 'million': 1, 'innoc': 1, 'men': 1, 'women': 1, 'children': 1, 'sinc': 1, 'introduct': 1, 'burnt': 1, 'tortur': 1, 'fine': 1, 'imprison': 1, 'what': 1, 'effect': 1, 'thi': 1, 'coercion': 1, 'make': 1, 'world': 1, 'fool': 1, 'other': 1, 'hypocrit': 1, 'support': 1, 'error': 1, 'over': 1, 'earth': 1, 'six': 1, 'histor': 1, 'american': 1, 'john': 1, 'e': 1, 'remsburg': 1, 'letter': 1, 'william': 1, 'short': 1, 'again': 1, 'becom

In [217]:
from scipy.sparse import csr_matrix

class WordCounterToVectorTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, vocabulary_size=1000, cap_count=10):
    self.vocabulary_size = vocabulary_size
    self.cap_count = cap_count

  def fit(self, X, y=None):
    total_count = Counter()
    for word_count in X:
      for word, count in word_count.items():
        total_count[word] += min(count, self.cap_count) # cap some words contribution of each email by 10

    most_common = total_count.most_common()[:self.vocabulary_size] # select most common words to be the vocabulary
    self.vocabulary_ = {word: index + 1
                        for index, (word, count) in enumerate(most_common)} # leave first place in vocabulary for later
    return self

  def transform(self, X, y=None):
    rows=[]
    cols=[]
    data=[]
    for row, word_count in enumerate(X):
      for word, count in word_count.items():
        rows.append(row)
        cols.append(self.vocabulary_.get(word, 0))
        data.append(count)
    return csr_matrix((data, (rows, cols)),
                      shape=(len(X), self.vocabulary_size + 1))


In [218]:
vocab_transformer = WordCounterToVectorTransformer(vocabulary_size=10)
X_few_vectors = vocab_transformer.fit_transform(X_few_wordcounts)
X_few_vectors

<3x11 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [219]:
from sklearn.pipeline import Pipeline

preprocessing = Pipeline([
    ("email_to_word_count",EmailToWordCounterTransformer()),
    ("word_count_to_vector", WordCounterToVectorTransformer())
])

X_train_transformed = preprocessing.fit_transform(X_train)

In [220]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

log_clf = LogisticRegression(max_iter=1000, random_state=42)
preds = cross_val_predict(log_clf, X_train_transformed, y_train, cv=3)

In [221]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

acc = accuracy_score(preds, y_train)
prec = precision_score(preds, y_train)
rec = recall_score(preds, y_train)
f1 = f1_score(preds, y_train)

print(f"Accuracy: {acc:.2%}")
print(f"Precision: {prec:.2%}")
print(f"Recall: {rec:.2%}")
print(f"F1: {f1:.2%}")

Accuracy: 98.50%
Precision: 92.84%
Recall: 98.17%
F1: 95.43%


In [222]:
X_test_transformed = preprocessing.transform(X_test)

log_clf = LogisticRegression(max_iter=1000, random_state=42)
log_clf.fit(X_train_transformed, y_train)

y_pred = log_clf.predict(X_test_transformed)

print(f"Precision: {precision_score(y_test, y_pred):.2%}")
print(f"Recall: {recall_score(y_test, y_pred):.2%}")

Precision: 95.88%
Recall: 97.89%


# Hard ham

In [223]:
ham_dir, spam_dir = fetch_spam_data(easy=False)
ham_filenames = [f for f in sorted(ham_dir.iterdir()) if len(f.name) > 20]
spam_filenames = [f for f in sorted(spam_dir.iterdir()) if len(f.name) > 20]
ham_emails = [load_email(filepath) for filepath in ham_filenames]
spam_emails = [load_email(filepath) for filepath in spam_filenames]

X = np.array(ham_emails + spam_emails, dtype=object)
y = np.array([0] * len(ham_emails) + [1] * len(spam_emails)) # zeros for ham and ones for spam

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [224]:
print(ham_emails[0].get_content().strip()[:500])

======================== THE MOTLEY FOOL ========================
                        PERSONAL  FINANCE 
                    Wednesday, January 2, 2002 
										 
mkettler@home.com

IN THIS ISSUE

- ASK THE FOOL: Stop the Solicitation!
  http://www.fool.com/m.asp?i=562064

- YOUR MONEY: Resolutions You Can Keep
  http://www.fool.com/m.asp?i=562065

- SPOTLIGHT: Should All My Savings Be in the Market?
  http://www.fool.com/m.


In [225]:
print(spam_emails[0].get_content().strip()[:500])

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN">
<HTML><HEAD>
<META content="text/html; charset=windows-1252" http-equiv=Content-Type>
<META content="MSHTML 5.00.2314.1000" name=GENERATOR></HEAD>
<BODY><!-- Inserted by Calypso -->
<TABLE border=0 cellPadding=0 cellSpacing=2 id=_CalyPrintHeader_ rules=none 
style="COLOR: black; DISPLAY: none" width="100%">
  <TBODY>
  <TR>
    <TD colSpan=3>
      <HR color=black noShade SIZE=1>
    </TD></TR></TD></TR>
  <TR>
    <TD colSpan=3>
   


In [226]:
def metrics(preds, y_train):
  acc = accuracy_score(preds, y_train)
  prec = precision_score(preds, y_train)
  rec = recall_score(preds, y_train)
  f1 = f1_score(preds, y_train)

  print(f"Accuracy: {acc:.2%}")
  print(f"Precision: {prec:.2%}")
  print(f"Recall: {rec:.2%}")
  print(f"F1: {f1:.2%}")

## Metrics for new dataset using the same Logistic regression

In [227]:
X_train_transformed = preprocessing.fit_transform(X_train)
X_test_transformed = preprocessing.fit_transform(X_test)

In [228]:
log_clf = LogisticRegression(max_iter=1000, random_state=42).fit(X_train_transformed, y_train)
preds = log_clf.predict(X_test_transformed)
metrics(preds, y_test)

Accuracy: 68.00%
Precision: 72.45%
Recall: 77.17%
F1: 74.74%


For easy ham metrics were:
Precision: 95.88%
Recall: 97.89%

## SVC

In [229]:
from sklearn.svm import SVC

svm_clf = SVC(random_state=42)
preds = cross_val_predict(svm_clf, X_train_transformed, y_train, cv=3)
metrics(preds, y_train)

Accuracy: 83.50%
Precision: 96.77%
Recall: 81.89%
F1: 88.71%


## Random forest

In [230]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(random_state=42)
preds = cross_val_predict(forest_clf, X_train_transformed, y_train, cv=3)
metrics(preds, y_train)

Accuracy: 92.17%
Precision: 97.51%
Recall: 91.38%
F1: 94.34%


## SGD

In [231]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
preds = cross_val_predict(sgd_clf, X_train_transformed, y_train, cv=3)
metrics(preds, y_train)

Accuracy: 80.67%
Precision: 75.62%
Recall: 94.41%
F1: 83.98%


## Gradient boosting

In [232]:
from sklearn.ensemble import GradientBoostingClassifier

gradient_clf = GradientBoostingClassifier(random_state=42)
preds = cross_val_predict(gradient_clf, X_train_transformed, y_train, cv=3)
metrics(preds, y_train)

Accuracy: 91.17%
Precision: 97.01%
Recall: 90.49%
F1: 93.64%


Logistic regression and Random forest seems to be the best ones, let's fine-tune them both

## Metrics for new dataset using the same Logistic regression

In [253]:
forest_clf = RandomForestClassifier(random_state=42).fit(X_train_transformed, y_train)
preds = forest_clf.predict(X_test_transformed)
metrics(preds, y_test)

Accuracy: 82.00%
Precision: 84.69%
Recall: 87.37%
F1: 86.01%


# Fine-tuning

## Preprocessing params fine-tuning

In [233]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

new_step = ("clf", LogisticRegression(max_iter=1000, random_state=42))
full_pipeline = Pipeline(preprocessing.steps + [new_step])

param_grid = [
    {
      'word_count_to_vector__vocabulary_size': randint(low=500, high=3000),
      'word_count_to_vector__cap_count': randint(low=5, high=20)
    }
]


rnd_search = RandomizedSearchCV(full_pipeline, param_grid, cv=3, n_iter=5, random_state=42, n_jobs=-1, scoring="f1")
rnd_search.fit(X_train, y_train)

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('email_to_word_count',
                                              EmailToWordCounterTransformer()),
                                             ('word_count_to_vector',
                                              WordCounterToVectorTransformer()),
                                             ('clf',
                                              LogisticRegression(max_iter=1000,
                                                                 random_state=42))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions=[{'word_count_to_vector__cap_count': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ba22bef0790>,
                                         'word_count_to_vector__vocabulary_size': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ba22bbe1790>}],
                   random_state=42, scoring='f1')

In [234]:
rnd_search.best_score_

0.9479441635204591

In [235]:
rnd_search.best_params_

{'word_count_to_vector__cap_count': 12,
 'word_count_to_vector__vocabulary_size': 2138}

We got new, a bit better params for our preprocessing step

In [236]:
from sklearn.base import clone

new_preprocessing = clone(preprocessing)
new_preprocessing.set_params(**rnd_search.best_params_)

Pipeline(steps=[('email_to_word_count', EmailToWordCounterTransformer()),
                ('word_count_to_vector',
                 WordCounterToVectorTransformer(cap_count=12,
                                                vocabulary_size=2138))])

In [254]:
X_train_transformed = new_preprocessing.fit_transform(X_train)

## Logistic regression

In [255]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_distribs = {"C": uniform(loc=0.001, scale=1000),
                  "solver": ["lbfgs", "liblinear"]}

logistic_clf = LogisticRegression(random_state=42, max_iter=1000)

rnd_search = RandomizedSearchCV(
    logistic_clf, param_distributions=param_distribs, n_iter=10, cv=3, random_state=42, scoring='f1'
)

rnd_search.fit(X_train_transformed, y_train)

RandomizedSearchCV(cv=3,
                   estimator=LogisticRegression(max_iter=1000, random_state=42),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7ba2282c2850>,
                                        'solver': ['lbfgs', 'liblinear']},
                   random_state=42, scoring='f1')

In [256]:
rnd_search.best_score_

0.9512068873868255

In [257]:
rnd_search.best_params_

{'C': 445.83375285359114, 'solver': 'lbfgs'}

In [258]:
best_logistic = LogisticRegression(random_state=42, max_iter=1000, **rnd_search.best_params_)

## Random forest

In [259]:
param_distribs = {
    "n_estimators": randint(100, 1000),
    "max_depth": randint(10, 100),
}

forest_clf = RandomForestClassifier(random_state=42)

rnd_search = RandomizedSearchCV(
    forest_clf, param_distributions=param_distribs, n_iter=10, cv=3, random_state=42, scoring='f1'
)

rnd_search.fit(X_train_transformed, y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ba22830b650>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7ba228308b90>},
                   random_state=42, scoring='f1')

In [260]:
rnd_search.best_score_

0.9463926668883872

In [261]:
rnd_search.best_params_

{'max_depth': 30, 'n_estimators': 714}

In [262]:
best_forest = RandomForestClassifier(random_state=42, **rnd_search.best_params_)

# Testing

In [263]:
X_test_transformed = new_preprocessing.fit_transform(X_test)

In [264]:
best_logistic.fit(X_train_transformed, y_train)
best_forest.fit(X_train_transformed, y_train)

RandomForestClassifier(max_depth=30, n_estimators=714, random_state=42)

In [265]:
logistic_preds = best_logistic.predict(X_test_transformed)
forest_preds = best_forest.predict(X_test_transformed)

### Logistic metrics

In [266]:
metrics(logistic_preds, y_test)

Accuracy: 72.67%
Precision: 68.37%
Recall: 87.01%
F1: 76.57%


Initial metrics for Logistic:

Accuracy: 68.00%
Precision: 72.45%
Recall: 77.17%
F1: 74.74%

With fine-tuned preprocessor:
Accuracy: 72.67%
Precision: 68.37%
Recall: 87.01%
F1: 76.57%

### Forest metrics

In [267]:
metrics(forest_preds, y_test)

Accuracy: 84.00%
Precision: 88.78%
Recall: 87.00%
F1: 87.88%


Initial metrics for Forest:

Accuracy: 82.00%
Precision: 84.69%
Recall: 87.37%
F1: 86.01%


Without fine-tuned preprocessing:

Accuracy: 84.67%
Precision: 88.78%
Recall: 87.88%
F1: 88.32%

With fine-tuned preprocessor - a bit worse because preprocessor was estimated by logistic regression predictions:

Accuracy: 84.00%
Precision: 88.78%
Recall: 87.00%
F1: 87.88%